We have fallen into RAM of the big North Pole computer. Read the whole stroy on the official [Day 17 of Advent of Code 2024](https://adventofcode.com/2024/day/17) website.

Here's a ChatGPT negotiated illustration of the setup:

![](chatgpt_image.webp){width=75% fig-align="center"}

::: {.callout-note collapse="true" title="Setting up"}

In [1]:
from misc.helper import verify_answer

example_input_file = "../inputs/example_day_18.txt"
input_file = "../inputs/day_18.txt"

:::

## Part 1: Escape the falling bits

Another grid problem! We are in a 2D grid where we need to figure out which positions of the grid are free from the falling bytes. 

In [13]:
from pathlib import Path
import numpy as np

def parse_input(input, grid_size=7, n_bytes=12):
    if Path(input).exists():
        with open(input) as file:
            input = file.read()
    
    input = input.strip().split("\n")
    grid = np.zeros((grid_size, grid_size), dtype=int)
    coord = [(int(y), int(x)) for row in input for x, y in [row.split(",")]]

    if n_bytes > len(coord):
        raise ValueError("n_bytes is larger than the number of coordinates")
    
    if max(coord, key=lambda x: x[0])[0] >= grid_size or max(coord, key=lambda x: x[1])[1] >= grid_size:
        raise ValueError("Coordinates are out of bounds of the grid")

    grid[tuple(zip(*coord[:n_bytes]))] = 1
    return grid

parse_input(example_input_file)

def print_grid_as_str(grid):
    # print 0 as ., 1 as # and 2 as O
    print("\n".join(["".join(["." if cell == 0 else "#" if cell == 1 else "O" for cell in row]) for row in grid]))

print_grid_as_str(parse_input(example_input_file))

...#...
..#..#.
....#..
...#..#
..#..#.
.#..#..
#.#....


Our task is to find the shortest path in the grid from the top left corner to the bottom right corner. We can only move up, down, left, or right, and we can't move diagonally. We can only move to a position if it is free from the falling bytes. And then we need to report the number of steps we need to take to reach the bottom right corner.

In [25]:
from collections import deque


def find_shortest_path(grid):
    n, m = grid.shape
    start = (0, 0)
    end = (n - 1, m - 1)

    def bfs(grid, start, end):
        queue = deque(
            [(start, [start])]
        )  # Queue contains tuples of (node, path_to_node)
        visited = set()

        while queue:
            node, path = queue.popleft()
            if node == end:
                return path  # Return the path as soon as we reach the end

            if node in visited:
                continue

            visited.add(node)
            x, y = node

            for dx, dy in (
                (1, 0),
                (-1, 0),
                (0, 1),
                (0, -1),
            ):  
                new_x, new_y = x + dx, y + dy
                if (
                    0 <= new_x < n
                    and 0 <= new_y < m
                    and grid[new_x, new_y] != 1
                ):
                    next_node = (new_x, new_y)
                    if next_node not in visited:
                        queue.append(
                            (next_node, path + [next_node])
                        ) 

        return None  

    return bfs(grid, start, end)

def part_one(input, grid_size=7, n_bytes=12):
    grid = parse_input(input, grid_size=grid_size, n_bytes=n_bytes)
    path = find_shortest_path(grid)
    return len(path) - 1

verify_answer(part_one, example_input_file, 22)

✔️ That's right! The answer is 22.


In [27]:
%time part_one(input_file, grid_size=71, n_bytes=1024)

CPU times: user 11.6 ms, sys: 2.26 ms, total: 13.9 ms
Wall time: 20.4 ms


290

Part 2: When there is no escape!

Now we need to figure out at what point there is no escape.

In [ ]:
def check_if_path_exist(grid):
    n, m = grid.shape
    start = (0, 0)
    end = (n - 1, m - 1)

    def bfs(grid, start, end):
        queue = deque(
            [(start, [start])]
        )  # Queue contains tuples of (node, path_to_node)
        visited = set()

        while queue:
            node, path = queue.popleft()
            if node == end:
                return path  # Return the path as soon as we reach the end

            if node in visited:
                continue

            visited.add(node)
            x, y = node

            for dx, dy in (
                (1, 0),
                (-1, 0),
                (0, 1),
                (0, -1),
            ):  
                new_x, new_y = x + dx, y + dy
                if (
                    0 <= new_x < n
                    and 0 <= new_y < m
                    and grid[new_x, new_y] != 1
                ):
                    next_node = (new_x, new_y)
                    if next_node not in visited:
                        queue.append(
                            (next_node, path + [next_node])
                        ) 

        return None  

    return bfs(grid, start, end) is not None